### Import libraries

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

ModuleNotFoundError: No module named 'mplcursors'

### Load data

In [ ]:
#experiments = [7, 16, 45]
#experiments = [0, 1000, 1001, 1002, 1003, 1004]
#experiments = [7]#, 16]
#experiments = [1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016] # Lower cars
experiments = [1014] # Communal reward
#experiments = [1017, 1018, 1019, 1020] # Increase cars to charger ratio
#experiments = [477, 478, 479] # Cumulative reward experiments
experiments = [1021, 1022, 1023, 1024] # Increase cars to charger ratio and use communal reward
#experiments = [1025, 1026] # 2500 eps using 5 zones and using only zones 2 and 4 (same as 1011 otherwise)

exp_agent_data = []

for exp_num in experiments:
    config_fname = f'../Exp_{exp_num}/config.yaml'
    
    c = load_config_file(config_fname)
    nn_c = c['nn_hyperparameters']
    federated_c = c['federated_learning_settings']
    algo_c = c['algorithm_settings']
    env_c = c['environment_settings']
    eval_c = c['eval_config']
    
    verbose = eval_c['verbose']
    
    if eval_c['fixed_attributes'] != [0, 1] and eval_c['fixed_attributes'] != [1, 0] and eval_c['fixed_attributes'] != [0.5, 0.5]:
        attr_label = 'learned'
    else:
        fixed_attributes = eval_c['fixed_attributes']
        attr_label = f'{fixed_attributes[0]}_{fixed_attributes[1]}'
    
    current_datetime = datetime.now()
    date = current_datetime.strftime('%Y-%m-%d_%H-%M')

    seed = env_c['seed']

    algorithm_dm = algo_c['algorithm']
    
    def load_from_json_with_error_handling(filepath):
        try:
            return read_csv_data(filepath, columns=['episode', 'timestep', 'done', 'zone', 'aggregation', 'agent_index', 'car_model', 'reward'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {filepath}: {e.msg} at line {e.lineno}, column {e.colno}")
            return None  # Handle the error and return None or an empty object
    
    d_base = f"../../../../../storage_1/metrics/Exp_{exp_num}"
    
    if not os.path.exists(d_base):
        d_base = f"../metrics/Exp_{exp_num}"
            
    base_path = f"{d_base}/train/metrics"

    print(f'Loading {base_path}_agent_metrics.csv')
    agent_data = load_from_json_with_error_handling(f'{base_path}_agent_metrics.csv')
    agent_data['seed'] = seed
    agent_data['exp_num'] = exp_num
    agent_data['algorithm'] = algorithm_dm
    agent_data['season'] = env_c['season']

    exp_agent_data.append(agent_data)

### Modify reward data to be cumulative averages

In [ ]:
# Convert data to DataFrame for easier manipulation
df = pd.concat(exp_agent_data, ignore_index=True)

# Scale timestep rewards by timestep index (uncomment this for reward v2)
#df['reward'] = df['reward'] / (df['timestep'] + 1)

# Only show select timesteps
#df = df[df['timestep'] >= 2]

# Create a new DataFrame that just has the cumulative reward per episode, zone, aggregation, agent_index, and seed
cumulative_reward_df = df.groupby(['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season'])['reward'].sum().reset_index()

# Rename the 'reward' column to 'cumulative_reward' for clarity
cumulative_reward_df.rename(columns={'reward': 'cumulative_reward'}, inplace=True)

# Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
cumulative_reward_df['episode'] = cumulative_reward_df['aggregation'] * nn_c['num_episodes'] + cumulative_reward_df['episode']

# **Sort the DataFrame to ensure correct order for expanding mean**
cumulative_reward_df.sort_values(['zone', 'agent_index', 'seed', 'episode'], inplace=True)

# **Calculate cumulative average reward per episode, zone, agent_index, and seed**
cumulative_reward_df['cumulative_avg_reward'] = cumulative_reward_df.groupby(
    ['zone', 'agent_index', 'seed']
)['cumulative_reward'].expanding().mean().reset_index(level=[0, 1, 2], drop=True)


# Calculate cumulative average reward per episode and zone
cumulative_avg_reward_by_zone = cumulative_reward_df.groupby(['episode', 'zone', 'seed'])['cumulative_reward'].mean().reset_index()
cumulative_avg_reward_by_zone['cumulative_reward'] = cumulative_avg_reward_by_zone.groupby(['zone', 'seed'])['cumulative_reward'].expanding().mean().reset_index(level=[0, 1], drop=True)

# Calculate cumulative average reward per episode and experiment number
cumulative_avg_reward_by_exp_num = cumulative_reward_df.groupby(['episode', 'exp_num', 'seed'])['cumulative_reward'].mean().reset_index()
cumulative_avg_reward_by_exp_num['cumulative_reward'] = cumulative_avg_reward_by_exp_num.groupby(['exp_num', 'seed'])['cumulative_reward'].expanding().mean().reset_index(level=[0, 1], drop=True)

# Calculate cumulative average reward per episode and algorithm
cumulative_avg_reward_by_algorithm = cumulative_reward_df.groupby(['episode', 'algorithm', 'seed'])['cumulative_reward'].mean().reset_index()
cumulative_avg_reward_by_algorithm['cumulative_reward'] = cumulative_avg_reward_by_algorithm.groupby(['algorithm', 'seed'])['cumulative_reward'].expanding().mean().reset_index(level=[0, 1], drop=True)

# Calculate cumulative average reward per episode and algorithm and season
cumulative_avg_reward_by_algorithm_season = cumulative_reward_df.groupby(
    ['episode', 'algorithm', 'season', 'seed']
)['cumulative_reward'].mean().reset_index()
cumulative_avg_reward_by_algorithm_season = cumulative_avg_reward_by_algorithm_season.sort_values(
    ['algorithm', 'season', 'seed', 'episode']
)
cumulative_avg_reward_by_algorithm_season['cumulative_avg_reward'] = cumulative_avg_reward_by_algorithm_season.groupby(
    ['algorithm', 'season', 'seed']
)['cumulative_reward'].transform(lambda x: x.expanding().mean())

# Calculate cumulative average reward per episode and season
cumulative_avg_reward_by_season = cumulative_reward_df.groupby(['episode', 'season', 'seed'])['cumulative_reward'].mean().reset_index()
cumulative_avg_reward_by_season['cumulative_reward'] = cumulative_avg_reward_by_season.groupby(['season', 'seed'])['cumulative_reward'].expanding().mean().reset_index(level=[0, 1], drop=True)

### PLOT DATA

# **Prepare data for plotting by aggregating over seeds and agent indices**
plot_data = cumulative_reward_df.groupby(['episode', 'zone']).agg({
    'cumulative_avg_reward': ['min', 'max', 'mean']
}).reset_index()
plot_data.columns = ['episode', 'zone', 'min_reward', 'max_reward', 'mean_reward']

# Plot the cloud plot for each zone
plt.figure(figsize=(8, 6))
for zone in plot_data['zone'].unique():
    # Filter the data for the current zone
    zone_data = plot_data[plot_data['zone'] == zone]
    
    plt.fill_between(
        zone_data['episode'], 
        zone_data['min_reward'], 
        zone_data['max_reward'], 
        alpha=0.3
    )
    plt.plot(
        zone_data['episode'], 
        zone_data['mean_reward'], 
        label=f'Zone {zone} Average Reward'
    )

plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title('Reward by Zone and Agent Index')

plt.legend()
plt.grid(True)
plt.show()

### OLD PLOT DATA

# Plot the cloud plot for each zone
plt.figure(figsize=(8, 6))
for zone in df['zone'].unique():
    # Filter the data for the current zone
    zone_data = cumulative_avg_reward_by_zone.loc[cumulative_avg_reward_by_zone['zone'] == zone]
    min_cumulative_avg_reward = zone_data.groupby('episode')['cumulative_reward'].min()
    max_cumulative_avg_reward = zone_data.groupby('episode')['cumulative_reward'].max()
    mean_cumulative_avg_reward = zone_data.groupby('episode')['cumulative_reward'].mean()

    plt.fill_between(
        min_cumulative_avg_reward.index, 
        min_cumulative_avg_reward.values, 
        max_cumulative_avg_reward.values, 
        alpha=0.3
    )
    plt.plot(
        mean_cumulative_avg_reward.index, 
        mean_cumulative_avg_reward.values, 
        label=f'Zone {zone} Average Reward'
    )

plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title(f'Aggs. {federated_c["aggregation_count"]} - Eps. per agg. {nn_c["num_episodes"]} - Cumulative Average Reward per Episode by Zone and Seed')
#plt.title(f'Version 2 Reward Function')

plt.legend()
plt.grid(True)
plt.show()


# Plot the cloud plot for each experiment
plt.figure(figsize=(8, 6))
for exp_num in df['exp_num'].unique():
    # Filter the data for the current zone
    exp_data_data = cumulative_avg_reward_by_exp_num.loc[cumulative_avg_reward_by_exp_num['exp_num'] == exp_num]
    mean_cumulative_avg_reward = exp_data_data.groupby('episode')['cumulative_reward'].mean()
    plt.plot(
        mean_cumulative_avg_reward.index, 
        mean_cumulative_avg_reward.values, 
        label=f'Experiment {exp_num} Reward'
    )

plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title(f'Aggs. {federated_c["aggregation_count"]} - Eps. per agg. {nn_c["num_episodes"]} - Cumulative Average Reward per Episode by Experiment Number and Seed')
plt.legend()
plt.grid(True)
plt.show()


# Plot the cloud plot for each algorithm
plt.figure(figsize=(8, 6))
for algo in df['algorithm'].unique():
    # Filter the data for the current zone
    algo_data = cumulative_avg_reward_by_algorithm.loc[cumulative_avg_reward_by_algorithm['algorithm'] == algo]
    min_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].min()
    max_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].max()
    mean_cumulative_avg_reward = algo_data.groupby('episode')['cumulative_reward'].mean()

    plt.fill_between(
        min_cumulative_avg_reward.index, 
        min_cumulative_avg_reward.values, 
        max_cumulative_avg_reward.values, 
        alpha=0.3
    )
    plt.plot(
        mean_cumulative_avg_reward.index, 
        mean_cumulative_avg_reward.values, 
        label=f'Algorithm {algo} Average Reward'
    )

plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title(f'Aggs. {federated_c["aggregation_count"]} - Eps. per agg. {nn_c["num_episodes"]} - Cumulative Average Reward per Episode by Algorithm and Seed')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 8))  # Increased size for better visibility
for algo in cumulative_avg_reward_by_algorithm_season['algorithm'].unique():
    for season in cumulative_avg_reward_by_algorithm_season['season'].unique():
        # Filter the data for the current algorithm and season
        data = cumulative_avg_reward_by_algorithm_season.loc[
            (cumulative_avg_reward_by_algorithm_season['algorithm'] == algo) &
            (cumulative_avg_reward_by_algorithm_season['season'] == season)
        ]
        
        if data.empty:
            continue  # Skip if no data for this combination
        
        # Calculate min, max, and mean cumulative rewards per episode
        min_cumulative_avg_reward = data.groupby('episode')['cumulative_avg_reward'].min()
        max_cumulative_avg_reward = data.groupby('episode')['cumulative_avg_reward'].max()
        mean_cumulative_avg_reward = data.groupby('episode')['cumulative_avg_reward'].mean()
        
        # Plot the filled area between min and max
        plt.fill_between(
            min_cumulative_avg_reward.index, 
            min_cumulative_avg_reward.values, 
            max_cumulative_avg_reward.values, 
            alpha=0.3
        )
        
        # Plot the mean cumulative average reward
        plt.plot(
            mean_cumulative_avg_reward.index, 
            mean_cumulative_avg_reward.values, 
            label=f'Algorithm {algo} Season {season} Average Reward'
        )

# Customize the plot
plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title(
    f'Aggs. {federated_c["aggregation_count"]} - '
    f'Eps. per agg. {nn_c["num_episodes"]} - '
    f'Cumulative Average Reward per Episode by Algorithm and Season'
)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Place legend outside the plot
plt.grid(True)
plt.tight_layout()  # Adjust layout to accommodate legend
plt.show()

# Plot the cloud plot for each season
plt.figure(figsize=(8, 6))
for season in df['season'].unique():
    # Filter the data for the current zone
    season_data = cumulative_avg_reward_by_season.loc[cumulative_avg_reward_by_season['season'] == season]
    min_cumulative_avg_reward = season_data.groupby('episode')['cumulative_reward'].min()
    max_cumulative_avg_reward = season_data.groupby('episode')['cumulative_reward'].max()
    mean_cumulative_avg_reward = season_data.groupby('episode')['cumulative_reward'].mean()

    plt.fill_between(
        min_cumulative_avg_reward.index, 
        min_cumulative_avg_reward.values, 
        max_cumulative_avg_reward.values, 
        alpha=0.3
    )
    plt.plot(
        mean_cumulative_avg_reward.index, 
        mean_cumulative_avg_reward.values, 
        label=f'Season {season} Average Reward'
    )

plt.xlabel('Episode')
plt.ylabel('Cumulative Average Reward')
plt.title(f'Aggs. {federated_c["aggregation_count"]} - Eps. per agg. {nn_c["num_episodes"]} - Cumulative Average Reward per Episode by Season and Seed')
plt.legend()
plt.grid(True)
plt.show()